# Problem 1. Mushroom Weight and Height

In the class we build a naive bayes classifier which classify whether a mushroom is poisonous or edible. In the class, all the feature are of categorical type; it has a discrete number of outcome as opposed to real number.

In this problem we want to explore two ways to deal with real number features for Naive Bayes classifier.

## The Data
The data given to you is very similar to mushroom data. It has three features: cap-color(with the same dictionary we did in class), mushroom-weight(made up by me), mushroom-height(also made up by me).

The data is given in
`mushrooms_homework_test.csv`
and
`mushrooms_homework_train.csv`

## Task 1.1) Simplest way. Just bin it.

The simplest way for dealing with continuous value feature is to discretize it. The simplest way to discretize is just to bin it. For example if your data looks like

`data = (0.9, 1.1, 1.2, 2.1, 2.2, 4.2, 5.3, 5.1)`

We count bin it with bin edges of 

`bins = (1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5)`

the bin number of a data point $x$ is the index $i$ where `bins[i-1] < x < bins[i]`. If $x$ is less than the minimum of bin edge then the bin number is 0. If $x$ is more than the maximum of bin edge then it's `len(bins)`.

Ex the data points above would be turned into
`binno = (0, 1, 1, 3, 3, 7, 9, 9)`

Once we discretize the value we can just use Bayes Classifier we did in class.

**Your task is to build a naive bayes classifier with binned height and binned weight. Pick appropriate bin edges somehow.
Then test your algorithm on the test data set. Report how many you got right and wrong in test data**


In [28]:
import pandas as pd
import numpy as np

In [29]:
df = pd.read_csv('data/mushrooms_homework_train.csv')
df

,xclass,cap_color,weight,height
0,e,y,6.122458,7.143689
1,e,w,4.709259,7.398728
2,p,w,2.341551,4.733059
3,e,g,3.954025,4.040427
4,e,y,3.456429,6.422466
...,...,...,...,...
6502,p,n,3.103014,3.211495
6503,e,n,5.162033,6.841161
6504,e,n,4.754516,5.347342
6505,e,n,3.272145,8.031833


In [31]:
def make_bin_column(col_data):
    bins = np.linspace(np.min(col_data), np.max(col_data), 20)
    return np.digitize(col_data, bins=bins)

df['dheight'] = make_bin_column(df.height)
df['dweight'] = make_bin_column(df.weight)

In [32]:
features = df[['cap_color', 'dweight', 'dheight']]
xclasses = df['xclass']

In [76]:
from typing import Dict
def max_value_key(d: Dict[str, float])->str:
    best_key = None
    best_value = 0.
    first = True
    for k, v in d.items():
        if first or v>best_value:
            best_key = k
            best_value = v
            first=False
    return best_key
    

In [79]:
from dataclasses import dataclass

@dataclass(frozen=True)
class ProbKey:
    fname: str
    value: str
    cls: str
    

class NaiveBayes:
    def __init__(self):
        # self.prob_dict (feature_name, value, classname) -> 
        self.prob_dict = {}
        self.fnames = []
        self.all_classes = []
        self.prior = {}
        
    def _cal_prob(self, features, xclasses, fname, value, cls):
        value_mask = features[fname] == value
        cls_mask = xclasses == cls
        n_right = np.sum(value_mask & cls_mask)
        n_cls = sum(cls_mask)
        prob = n_right / n_cls
        return prob 
        
    
    def train(self, features, xclasses):
        all_classes = xclasses.unique()
        self.fnames = features.columns
        self.all_classes = all_classes
        prob_dict = {}
        self.prior = {cls:sum(xclasses==cls)/len(xclasses) for cls in all_classes}
        
        for fname in features.columns:
            for value in features[fname].unique():
                for cls in all_classes:
                    key = ProbKey(fname=fname, value=value, cls=cls)
                    prob = self._cal_prob(features, xclasses, fname, value, cls)
                    prob_dict[key] = prob
        self.prob_dict = prob_dict
    
    def predict_top_one(self, data, cls: str):
        p = self.prior[cls]
        for fname in self.fnames:
            value = getattr(data, fname)
            key = ProbKey(fname, value, cls)
            this_p = self.prob_dict[key]
            p *= this_p
        return p #just the prior*Prod P(x|cls)
            
    def predict_prob(self, data):
        top_dict = {cls: self.predict_top_one(data, cls) for cls in self.all_classes}
        evidence = sum(v for k,v in top_dict.items())
        return {cls: v/evidence for cls, v in top_dict.items()}
    
    def predict_class(self, data):
        #return cls with highest prob
        probs = self.predict_prob(data)
        best_cls = max_value_key(probs)
        return best_cls


In [80]:
nb = NaiveBayes()
nb.train(features, xclasses)
# for data, xclass in zip(features.itertuples(), xclasses):
#     print(nb.predict_prob(data), xclass)

In [81]:
test_df = pd.read_csv('data/mushrooms_homework_test.csv')
test_df['dheight'] = make_bin_column(test_df.height)
test_df['dweight'] = make_bin_column(test_df.weight)

test_features = test_df[['cap_color', 'dweight', 'dheight']]
test_xclasses = test_df['xclass']

In [82]:
correct = 0
total = 0
for data, xclass in zip(features.itertuples(), xclasses):
    if nb.predict_class(data) == xclass:
        correct+=1
    total += 1
print(correct, total, correct/total)

4771 6507 0.7332103888120486


## Task 1.2) Gaussian Naive Bayes.

We could assume a certain probability distribution function(pdf) for the conditional probability. One popular choice is normal distrubution/gaussian distribution.

Let us understand this assumption intuitively. The idea is that the *weight* of *poisonous* mushroom is normally distributed around some mean with some width while the *weight* for edible mushroom is hopefully normally distributed around some other mean.

Let us say that the weight of poisonous mushroom is normally distributed around $2\pm1$ gram(I made up this number)  while the edible mushroom is normally distributed at $5\pm 1$ gram. If we found a mushroom of 2.5 gram. It's probably the poisonous one since edible one should weight around 5 gram.

<img src="gaussian.png" width="400"/>

Concretely, we assume that the probability distribution of feature $X$ given that it is of class $y$ to be

$$
\displaystyle
pdf(X=x|y) = \frac{1}{\sqrt{2\pi}} \exp\left[\frac{-(x-\mu_y)^2}{2\sigma_y^2}\right]
$$

 - $\mu_y$ is the mean of feature $X$ given that it is of class $y$. Ex: mean weight($X$) of poisonous mushroom (eg: 2 gram).

 - $\sigma_y$ is the standard deviation of feature $X$ given that it is of class $y$. Ex std. dev. of weight of poisonous mushroom(eg: \pm 1 gram)
 
Recall the relatio between pdf and probability.
$$
    P(X \in (x, x+\delta x)|y) = \int^{x+\delta x}_x pdf(X=x) \;\text{d}x
$$

for small enough $\delta x$ it becomes
$$
    P(X \in (x, x+\delta x)|y) = pdf(X=x) \delta x
$$

Now here is the important part. From the above $P(X=x|y)$ and $P(X=x|\sim y)$ has a factor of $\delta x$(I drop of the range for brevity). This means that the factor of $\delta x$ appear in both the numerator and denominator of the formula we use for calculating probability. Thus the $\delta x$ cancels out nicely. This means that **we can just use pdf(X=x|y) as P(X=x|y)** in naive bayes formula we got and every thing will just work out. We don't have to worry about the $\delta x$ part

**Your task: build a classifier similar to what you did in 1.1. Except now you use gaussian distribution assumption for the continous features. Measure your performance against the test data**

In [86]:
from typing import List
def gaussian(x, mu, sigma):
    return 1/np.sqrt(2*np.pi)*np.exp(-(x-mu)**2/2/sigma**2)

In [106]:
@dataclass(frozen=True)
class GaussianParam:
    mu: float
    sigma: float

@dataclass(frozen=True)
class GaussianKey:
    fname: str
    cls: str
    
class NaiveBayesGaussian:
    def __init__(self):
        # self.prob_dict (feature_name, value, classname) ->
        self.prob_dict = {}
        self.all_classes = []
        self.prior = {}
        self.categorical_features = []  # list of categorical features
        self.continuous_features = []  # list of continuous features
        self.gaussian_param = {}  # dict from continuous featurename -> Gaussian param

    def _cal_prob(self, features, xclasses, fname, value, cls):
        value_mask = features[fname] == value
        cls_mask = xclasses == cls
        n_right = np.sum(value_mask & cls_mask)
        n_cls = sum(cls_mask)
        prob = n_right / n_cls
        return prob

    def train(self, features, xclasses, continuous_features: List[str] = None):
        if continuous_features is None:
            continuous_features = []

        self.categorical_features = [
            col for col in features.columns if col not in continuous_features]
        self.continuous_features = continuous_features
        all_classes = xclasses.unique()
        self.all_classes = all_classes
        prob_dict = {}
        self.prior = {cls: sum(xclasses == cls)/len(xclasses)
                      for cls in all_classes}

        for fname in self.categorical_features:
            for value in features[fname].unique():
                for cls in all_classes:
                    key = ProbKey(fname=fname, value=value, cls=cls)
                    prob = self._cal_prob(
                        features, xclasses, fname, value, cls)
                    prob_dict[key] = prob
        self.prob_dict = prob_dict

        # for continuous one we use the gaussian
        def get_gaussian_features(col, cls):
            feature = features[col][xclasses==cls]
            return GaussianParam(mu=np.mean(feature), sigma=np.std(feature))
        self.gaussian_param = {GaussianKey(fname=col, cls=cls):get_gaussian_features(col, cls) 
                               for col in self.continuous_features
                               for cls in self.all_classes}
        

    def predict_top_one(self, data, cls: str):
        p = self.prior[cls]
        for fname in self.categorical_features:
            value = getattr(data, fname)
            key = ProbKey(fname, value, cls)
            this_p = self.prob_dict[key]
            p *= this_p
        
        for fname in self.continuous_features:
            gp = self.gaussian_param[GaussianKey(fname=fname, cls=cls)]
            value = getattr(data, fname)
            p *= gaussian(value, gp.mu, gp.sigma)
        return p  # just the prior*Prod P(x|cls)

    def predict_prob(self, data):
        top_dict = {cls: self.predict_top_one(
            data, cls) for cls in self.all_classes}
        evidence = sum(v for k, v in top_dict.items())
        return {cls: v/evidence for cls, v in top_dict.items()}

    def predict_class(self, data):
        # return cls with highest prob
        probs = self.predict_prob(data)
        best_cls = max_value_key(probs)
        return best_cls

In [107]:
df = pd.read_csv('data/mushrooms_homework_train.csv')
df

,xclass,cap_color,weight,height
0,e,y,6.122458,7.143689
1,e,w,4.709259,7.398728
2,p,w,2.341551,4.733059
3,e,g,3.954025,4.040427
4,e,y,3.456429,6.422466
...,...,...,...,...
6502,p,n,3.103014,3.211495
6503,e,n,5.162033,6.841161
6504,e,n,4.754516,5.347342
6505,e,n,3.272145,8.031833


In [108]:
features = df[['cap_color', 'weight', 'height']]
xclasses = df.xclass
nb = NaiveBayesGaussian()
nb.train(features, xclasses, continuous_features = ['weight', 'height'])

In [110]:
nb.gaussian_param

{GaussianKey(fname='weight', cls='e'): GaussianParam(mu=4.310016666234858, sigma=1.2925952634849398),
 GaussianKey(fname='weight', cls='p'): GaussianParam(mu=3.2141575420054664, sigma=0.9972401361343372),
 GaussianKey(fname='height', cls='e'): GaussianParam(mu=5.995639636920748, sigma=1.9003781732257048),
 GaussianKey(fname='height', cls='p'): GaussianParam(mu=5.00334707527546, sigma=1.4240001555328947)}

In [111]:
df.weight[df.xclass=='p'].mean()

3.2141575420054664

In [112]:
correct = 0
total = 0
for data, xclass in zip(features.itertuples(), xclasses):
    if nb.predict_class(data) == xclass:
        correct+=1
    total += 1
print(correct, total, correct/total)

4691 6507 0.7209159366835716


# Problem 2. Product Reviews

Naive Bayes is quite powerful given its simplicity. Typically the usefulness of a Machine learning Algorithm is limited only by your imgination on what to ask. If you ask an interesting question, you got a useful system. If you ask a dump question, you got a useless system.

In this problem we will explore a text mining application using Naive Bayes.

The goal of this problem is to make a system that can read customer review and tells whether the customer recommend the product to others or not.

The data is stolen from https://www.kaggle.com/nicapotato/womens-ecommerce-clothing-reviews
I splitted it into train(`clothing_reviews_train.csv`) and test(`clothing_reviews_test.csv`) for you.

The two columns that is relavant to this problem are.
- Recommended IND
- Title
- Review Text

**Do not use any other column**

You could do challenging version (No extra point except for bragging rights)
Use the data from http://jmcauley.ucsd.edu/data/amazon/ and do similar thing --> the book review is hugeeeeee


## Your task
Build a classifier which you can give it a reviewtext and review title and it can split out whether the reviewer recommend the product or not. Measure the performance on test data `clothing_reviews_test.csv`.

## Some Guide for you.

- Be sure to normalize your text. Example [here](https://machinelearningmastery.com/clean-text-machine-learning-python/). This includes
    - lowercase everything
    - clean out stop words
    - get rid of punctuations
    - stem the word
    - Yes you may use nltk only for cleaning up part.
- Be careful as you are multiplying a whole bunch of small numbers together. You are better off adding the log and exponentiate it back.
- Read up lecture notes on spam filtering. Especially on how to deal with missing words. You can read up [old version of excercise 1 from sam](https://github.com/KongsakTi/PatternReg/tree/master/Exercise%201)
- **Do not** get stuck on this alone. Find help/Consult your friends if you are stuck. Collaboration is allowed but you must understand what you send in.

In [1]:
#you are now on your own!!!